In [12]:
import os
import json

import pandas as pd

In [13]:
class JsonConvertDf():
    def __init__(self, json_path:str, root_name:str='default'):
        self.json_path = json_path
        self.json_data = None
        self.root_name = root_name
        self.df = None

        with open(json_path, "rb") as f:
            self.json_data = json.load(f)

    def run(self)->pd.DataFrame:
        json_data = self.json_data
        if type(self.json_data)== list:
            new_obj = {}
            for index, value in enumerate(self.json_data):
                new_obj[str(index)] = value

            json_data = new_obj

        rapper_json = { self.root_name:json_data }
        
        json_normal = pd.json_normalize(rapper_json, sep='_')

        self.df = json_normal

        return self.df

    
class JsonConvertDfChild(JsonConvertDf):
    def __init__(self, json_path:str, root_name:str, max_column:int=200):
        self.max_column = max_column
        super().__init__(json_path, root_name)

    def run(self)->pd.DataFrame:
        self.df = super().run()
        self.columns_count = len(self.df.columns)
        self.df = self.df[self.df.columns[:self.max_column]]
        return self.df

In [14]:
def structure_csv(src_path:str, dst_path:str)->bool:
    try:
        json_convert = JsonConvertDfChild(json_path=src_path, root_name='pikepdf', max_column=200)
        df = json_convert.run()
        df.to_csv(dst_path)
    except Exception as e:
        print(f'{str(e)}')
        return False

    if not os.path.isfile(dst_path):
        return False

    return True

In [15]:
src_path = '/data/outputs/00a9a1fd1a981859605c159ae848086edcc663875b8295818c10045fd616fc90.pikepdf.parser'
dst_path = '/data/outputs/00a9a1fd1a981859605c159ae848086edcc663875b8295818c10045fd616fc90.pikepdf.parser.feature'

In [21]:
os.path.isfile(src_path)

True

In [22]:
os.path.isfile(dst_path)

False

In [23]:
structure_csv(src_path, dst_path)

True

In [24]:
!cat $dst_path

,pikepdf_filename,pikepdf_Size,pikepdf_infolenth,pikepdf_PdfVersion,pikepdf_Pages,pikepdf_Author,pikepdf_Title,pikepdf_ModDate,pikepdf_CreationDate,pikepdf_Creator,pikepdf_Producer,pikepdf_len_ExtGState,pikepdf_len_Font,pikepdf_len_ProcSet,pikepdf_len_ColorSpace,pikepdf_len_XObject
0,00a9a1fd1a981859605c159ae848086edcc663875b8295818c10045fd616fc90,275,6,1.6,13,Administrator,,D:20170910122744+09'00',D:20170910122711+09'00',PScript5.dll Version 5.2.2,Acrobat Distiller 10.1.16 (Windows),1,5,3,1,6


In [25]:
df = pd.read_csv(dst_path)
df

,Unnamed: 0,pikepdf_filename,pikepdf_Size,pikepdf_infolenth,pikepdf_PdfVersion,pikepdf_Pages,pikepdf_Author,pikepdf_Title,pikepdf_ModDate,pikepdf_CreationDate,pikepdf_Creator,pikepdf_Producer,pikepdf_len_ExtGState,pikepdf_len_Font,pikepdf_len_ProcSet,pikepdf_len_ColorSpace,pikepdf_len_XObject
0,0,00a9a1fd1a981859605c159ae848086edcc663875b8295...,275,6,1.6,13,Administrator,NaN,D:20170910122744+09'00',D:20170910122711+09'00',PScript5.dll Version 5.2.2,Acrobat Distiller 10.1.16 (Windows),1,5,3,1,6


In [ ]:
from pathlib import Path

OUT_DIR = '/data/outputs'

paths = Path(OUT_DIR).glob('*.pikepdf.parser')

for path in paths:
    src_path = str(path)
    dst_path = os.path.join(OUT_DIR, path.name + '.feature')
    
    print(f'{src_path} -> {dst_path}')
    structure_csv(src_path, dst_path)

/data/outputs/000a5028b6b95abd90b7cfa9070fd8a26222c0b0ede27621cebf1ae5a20b646a.pikepdf.parser -> /data/outputs/000a5028b6b95abd90b7cfa9070fd8a26222c0b0ede27621cebf1ae5a20b646a.pikepdf.parser.feature
/data/outputs/0016d894ce4686cbe7676635d66c23dbef29ccf7e3cf8378304baa33c0124b28.pikepdf.parser -> /data/outputs/0016d894ce4686cbe7676635d66c23dbef29ccf7e3cf8378304baa33c0124b28.pikepdf.parser.feature
/data/outputs/001c3b1a1a319653acf505c64550d559adf4afd2e7c6feaab5227df339d5c88f.pikepdf.parser -> /data/outputs/001c3b1a1a319653acf505c64550d559adf4afd2e7c6feaab5227df339d5c88f.pikepdf.parser.feature
/data/outputs/001ecf95455694bb36d00eb27014dd0d66c734040e32ec369c5abb03e3022288.pikepdf.parser -> /data/outputs/001ecf95455694bb36d00eb27014dd0d66c734040e32ec369c5abb03e3022288.pikepdf.parser.feature
/data/outputs/00245dfaa978a9d4b5ada5174449694bc07fdcedc046a0186bd35bc5f8436abd.pikepdf.parser -> /data/outputs/00245dfaa978a9d4b5ada5174449694bc07fdcedc046a0186bd35bc5f8436abd.pikepdf.parser.feature
/data

In [29]:
print(len(list(Path(OUT_DIR).glob('*.pikepdf.parser'))))
print(len(list(Path(OUT_DIR).glob('*.pikepdf.parser.feature'))))

10628
10628


# 다시 진행해야할 때 결과물 삭제하기. 조심해서 실행할 것...!!!!!!!!

In [ ]:
import glob
[os.remove(f) for f in glob.glob("/data/outputs/*.pikepdf.parser.feature")]